# Practical 2: Supply and use tables

## Learning goals
- Understand the structure of supply and use tables
- Practice and interpret the industry aggregation/disaggregation in SUT
- Practice how to transform SUTs into IOT

### Instructions
Download the supply and use tables of the Netherlands and place them in a data folder in your working directory

https://www.cbs.nl/en-gb/custom/2020/29/supply-and-use-input-output-and-sector-accounts.


In [1]:
import pandas as pd

### Exercise 1

1. Select two products for the year 2019
2. Can you please visualize in a bar graph the breakdown of final consumption?
3. Can you perform the same for gross fixed capital formation, value added and employment?  

### Exercise 2
1. In the use table, aggregate the industries into three sectors: primary sector, secondary sector, and
tertiary sector.
2. To which aggregated sector does your selected category belong?
3. How many products and services does your category supply? 

### Exercise 3
1. Following the instruction presented during the lecture implement the following two transformations methods from SUTs to IOT:
    - Industry Technology Assumption in Product-by-Product format
    - Industry Technology Assumption in Industry-by-Industry format